In [ ]:
import numpy as np
import copy


def add_score(score, value):
    return score + value

def compress_and_merge(row):
    non_zero = [val for val in row if val != 0]
    result = []
    score_gain = 0
    skip = False

    for i in range(len(non_zero)):
        if skip:
            skip = False
            continue
        if i + 1 < len(non_zero) and non_zero[i] == non_zero[i + 1]:
            merged = non_zero[i] * 2
            result.append(merged)
            score_gain += merged
            skip = True
        else:
            result.append(non_zero[i])

    result += [0] * (4 - len(result))
    return np.array(result), score_gain

def move_grid(grid, score, direction):
    updated_grid = grid.copy()
    total_score_gain = 0

    for i in range(4):
        if direction in ['left', 'right']:
            row = updated_grid[i, ::1 if direction == 'left' else -1]
            new_row, gained = compress_and_merge(row)
            updated_grid[i, ::1 if direction == 'left' else -1] = new_row
        else:
            col = updated_grid[:, i][::1 if direction == 'up' else -1]
            new_col, gained = compress_and_merge(col)
            updated_grid[:, i][::1 if direction == 'up' else -1] = new_col

        total_score_gain += gained

    return updated_grid, add_score(score, total_score_gain)

def add_new_tile(grid):
    empty = list(zip(*np.where(grid == 0)))
    if not empty:
        return False
    i, j = empty[np.random.randint(len(empty))]
    grid[i, j] = 2 if np.random.random() < 0.9 else 4
    return True

def check_win(grid):
    return 2048 in grid

def check_game_over(grid):
    if np.any(grid == 0):
        return False
    for i in range(4):
        for j in range(4):
            if (i < 3 and grid[i, j] == grid[i + 1, j]) or (j < 3 and grid[i, j] == grid[i, j + 1]):
                return False
    return True

def new_game():
    grid = np.zeros((4, 4), dtype=int)
    add_new_tile(grid)
    add_new_tile(grid)
    return grid, 0

def play_turn(grid, move, score):
    if check_game_over(grid):
        raise RuntimeError("GAME OVER")

    if move not in ['left', 'right', 'up', 'down']:
        raise ValueError("Invalid move")

    new_grid, new_score = move_grid(grid, score, move)

    if not np.array_equal(new_grid, grid):
        add_new_tile(new_grid)

    if check_win(new_grid):
        raise RuntimeError("YOU WIN")

    return new_grid, new_score

def print_grid(grid, score=None):
    if score is not None:
        print()
    print("+----" * 4 + "+")
    for row in grid:
        print("|" + "|".join(f"{num if num != 0 else '':4}" for num in row) + "|")
        print("+----" * 4 + "+")


def get_valid_move(grid, score):
    for move in ['up', 'left', 'right', 'down']:
        try:
            new_grid, _ = move_grid(grid.copy(), score, move)
            if not np.array_equal(new_grid, grid):
                return move
        except:
            continue
    return None

def run_simulation(num_games=30):
    results = {
        'scores': [],
        'wins': 0,
        'losses': 0,
        'max_tiles': [],
        'last_grid': None,
        'last_score': 0
    }

    for game in range(num_games):
        grid, score = new_game()

        while True:
            try:
                move = get_valid_move(grid, score)
                if move is None:
                    raise RuntimeError("GAME OVER")
                grid, score = play_turn(grid, move, score)
            except RuntimeError as e:
                if str(e) == "YOU WIN":
                    results['wins'] += 1
                elif str(e) == "GAME OVER":
                    results['losses'] += 1
                break

        results['scores'].append(score)
        results['max_tiles'].append(np.max(grid))

        if game == num_games - 1:
            results['last_grid'] = grid.copy()
            results['last_score'] = score

    return results

def print_statistics(results):
    scores = results['scores']
    max_tiles = results['max_tiles']
    last_score = results['last_score']
    last_grid = results['last_grid']

    print("After 30 games:")
    print(f"{'Best Score:':<18}{max(scores)}")
    print(f"{'Worst Score:':<18}{min(scores)}")
    print(f"{'Average Score:':<18}{sum(scores) / len(scores):.1f}")
    print(f"{'Wins:':<18}{results['wins']}")
    print(f"{'Losses:':<18}{results['losses']}")
    print(f"{'Average Max Tile:':<18}{sum(max_tiles) / len(max_tiles):.1f}")
    print(f"\nScore: {last_score}")
    print_grid(last_grid)



results = run_simulation(30)
print_statistics(results)


After 30 games:
Best Score:       6168
Worst Score:      716
Average Score:    2472.5
Wins:             0
Losses:           30
Average Max Tile: 202.7

Score: 1864
+----+----+----+----+
| 128|  64|  32|  16|
+----+----+----+----+
|  64|  32|  16|   8|
+----+----+----+----+
|  32|   8|   2|   4|
+----+----+----+----+
|   8|   2|   4|   2|
+----+----+----+----+
